In [2]:
# 1. Importar librerías necesarias
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt


In [3]:

# 2. Cargar el archivo CSV de los colegios
file_path = 'data/Colegios_a_nivel_nacional_20241014.csv'
df = pd.read_csv(file_path)

# Revisar las primeras filas para ver la estructura de los datos
df.head()
df.describe()


,AÑO,COD_DANE_DEPARTAMENTO,COD_SECRETARIA,COD_DANE_MUNICIPIO,CODIGO_DANE,COD_SECTOR,COD_CALENDARIO,FAX,TOTAL_MATRICULA,CANTIDAD_SEDES
count,21010.000000,21010.000000,21010.000000,21010.000000,2.101000e+04,21010.000000,21010.000000,6.999000e+03,21010.000000,21010.000000
mean,2018.823893,34.149833,4113.526226,34352.496192,2.985272e+11,1001.769491,58.574726,2.271018e+08,606.383484,1.486340
std,2.304990,26.682840,1172.252337,26713.508997,9.400833e+10,0.421169,82.643797,1.135290e+09,868.624865,1.857668
min,2015.000000,5.000000,3758.000000,5001.000000,1.050010e+11,1001.000000,52.000000,0.000000e+00,1.000000,1.000000
25%,2017.000000,11.000000,3766.000000,11001.000000,3.051480e+11,1002.000000,52.000000,2.894401e+06,96.000000,1.000000
50%,2019.000000,25.000000,3788.000000,25126.000000,3.170010e+11,1002.000000,52.000000,5.611035e+06,250.000000,1.000000
75%,2021.000000,68.000000,3818.000000,68001.000000,3.680010e+11,1002.000000,52.000000,7.437713e+06,700.750000,1.000000
max,2022.000000,97.000000,10930.000000,97001.000000,8.470580e+11,1002.000000,1110.000000,3.177225e+10,7141.000000,25.000000


In [4]:
# 3. Verificar si hay valores faltantes
df.isnull().sum()

# 4. Corregir el nombre de la columna si es necesario
df.columns = df.columns.str.replace('Ã‘O', 'AÑO')  # Corregir el nombre de la columna de año

# 5. Crear una columna auxiliar para clasificar el horario según el calendario
# Vamos a asumir que "Calendario A" es un tipo de horario y "Calendario B" es otro tipo de horario.
df['HORARIO'] = df['CALENDARIO'].apply(lambda x: 'Horario A' if x == 'A' else 'Horario B')

# 6. Agrupar los datos por departamento y tipo de horario
df_grouped = df.groupby(['DEPARTAMENTO', 'HORARIO']).size().reset_index(name='CANTIDAD')

# 7. Cargar el archivo GeoJSON de los departamentos de Colombia
colombia_shapefile = gpd.read_file('data/colombia.geo.json')

# Revisar las primeras filas del archivo GeoJSON
colombia_shapefile.head()


,DPTO,NOMBRE_DPT,AREA,PERIMETER,HECTARES,geometry
0,05,ANTIOQUIA,6.335186e+10,1963728.843,6335185.555,"POLYGON ((-76.30730 8.61930, -76.29810 8.61640..."
1,08,ATLANTICO,3.360765e+09,240936.172,336076.535,"POLYGON ((-74.87060 10.36120, -74.87640 10.343..."
2,11,SANTAFE DE BOGOTA D.C,1.650948e+09,323322.540,165094.778,"POLYGON ((-74.02290 4.79510, -74.02640 4.78360..."
3,13,BOLIVAR,2.614189e+10,1309427.968,2614189.453,"POLYGON ((-75.15950 10.42360, -75.15430 10.409..."
4,15,BOYACA,2.335258e+10,1364539.911,2335258.246,"POLYGON ((-72.21300 7.02750, -72.20610 7.02460..."


In [5]:

# 8. Unir los datos de los colegios con el shapefile usando el nombre del departamento
# Nos aseguramos de que los nombres de los departamentos coincidan entre los dos conjuntos de datos
merged = colombia_shapefile.set_index('NOMBRE_DPT').join(df_grouped.set_index('DEPARTAMENTO'))

# 9. Crear el mapa interactivo con folium
mapa = folium.Map(location=[4.570868, -74.297333], zoom_start=5)  # Coordenadas de Colombia

mapa


In [6]:
print(merged.columns)
print(colombia_shapefile.columns)



Index(['DPTO', 'AREA', 'PERIMETER', 'HECTARES', 'geometry', 'HORARIO',
       'CANTIDAD'],
      dtype='object')
Index(['DPTO', 'NOMBRE_DPT', 'AREA', 'PERIMETER', 'HECTARES', 'geometry'], dtype='object')


In [7]:
merged.head()

,DPTO,AREA,PERIMETER,HECTARES,geometry,HORARIO,CANTIDAD
NOMBRE_DPT,,,,,,,
ANTIOQUIA,05,6.335186e+10,1963728.843,6335185.555,"POLYGON ((-76.30730 8.61930, -76.29810 8.61640...",Horario A,1167.0
ANTIOQUIA,05,6.335186e+10,1963728.843,6335185.555,"POLYGON ((-76.30730 8.61930, -76.29810 8.61640...",Horario B,81.0
ATLANTICO,08,3.360765e+09,240936.172,336076.535,"POLYGON ((-74.87060 10.36120, -74.87640 10.343...",NaN,NaN
SANTAFE DE BOGOTA D.C,11,1.650948e+09,323322.540,165094.778,"POLYGON ((-74.02290 4.79510, -74.02640 4.78360...",NaN,NaN
BOLIVAR,13,2.614189e+10,1309427.968,2614189.453,"POLYGON ((-75.15950 10.42360, -75.15430 10.409...",NaN,NaN


In [17]:
df.describe()

,AÑO,COD_DANE_DEPARTAMENTO,COD_SECRETARIA,COD_DANE_MUNICIPIO,CODIGO_DANE,COD_SECTOR,COD_CALENDARIO,FAX,TOTAL_MATRICULA,CANTIDAD_SEDES
count,21010.000000,21010.000000,21010.000000,21010.000000,2.101000e+04,21010.000000,21010.000000,6.999000e+03,21010.000000,21010.000000
mean,2018.823893,34.149833,4113.526226,34352.496192,2.985272e+11,1001.769491,58.574726,2.271018e+08,606.383484,1.486340
std,2.304990,26.682840,1172.252337,26713.508997,9.400833e+10,0.421169,82.643797,1.135290e+09,868.624865,1.857668
min,2015.000000,5.000000,3758.000000,5001.000000,1.050010e+11,1001.000000,52.000000,0.000000e+00,1.000000,1.000000
25%,2017.000000,11.000000,3766.000000,11001.000000,3.051480e+11,1002.000000,52.000000,2.894401e+06,96.000000,1.000000
50%,2019.000000,25.000000,3788.000000,25126.000000,3.170010e+11,1002.000000,52.000000,5.611035e+06,250.000000,1.000000
75%,2021.000000,68.000000,3818.000000,68001.000000,3.680010e+11,1002.000000,52.000000,7.437713e+06,700.750000,1.000000
max,2022.000000,97.000000,10930.000000,97001.000000,8.470580e+11,1002.000000,1110.000000,3.177225e+10,7141.000000,25.000000


In [20]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim

# Crear una instancia del geocodificador
geolocator = Nominatim(user_agent="mi_aplicacion")

# Cargar los datos de municipios y departamentos
df_municipios = pd.read_csv('data/municipios_y_departamentos.csv')

# Cargar los datos de colegios
df_colegios = pd.read_csv('data/Colegios_a_nivel_nacional_20241014.csv')  # Asegúrate de cambiar esto al nombre correcto de tu archivo

def obtener_coordenadas_municipio(nombre_municipio):
    """
    Función para obtener las coordenadas (latitud y longitud) de un municipio
    dado su nombre.

    Parámetros:
    nombre_municipio (str): Nombre del municipio.

    Retorna:
    tuple: (latitud, longitud) del municipio.
    """
    try:
        location = geolocator.geocode(nombre_municipio + ', Colombia')
        if location:
            return location.latitude, location.longitude
        else:
            raise ValueError(f"No se encontraron coordenadas para el municipio: {nombre_municipio}")
    except Exception as e:
        print(f"Error al obtener coordenadas: {e}")
        return None, None

# Agrupar los datos de colegios por departamento y calendario
df_colegios['Colegios_Calendario_A'] = df_colegios['COD_CALENDARIO'].apply(lambda x: 1 if x == 52 else 0)
df_colegios['Colegios_Calendario_B'] = df_colegios['COD_CALENDARIO'].apply(lambda x: 1 if x != 52 else 0)

# Contar los colegios por departamento
df_departamento = df_colegios.groupby('COD_DANE_DEPARTAMENTO').agg({
    'Colegios_Calendario_A': 'sum',
    'Colegios_Calendario_B': 'sum',
    'TOTAL_MATRICULA': 'sum',
    'CANTIDAD_SEDES': 'sum'
}).reset_index()

# Unir los datos de departamentos con los datos de municipios
df_departamento = df_departamento.merge(df_municipios, how='left', left_on='COD_DANE_DEPARTAMENTO', right_on='CÓDIGO DANE DEL DEPARTAMENTO')

# Crear un mapa centrado en Colombia
mapa = folium.Map(location=[4.5709, -74.2973], zoom_start=6)

# Agregar marcador para cada departamento
for _, row in df_departamento.iterrows():
    # Obtener las coordenadas del departamento (usando el nombre del departamento)
    lat, lon = obtener_coordenadas_municipio(row['DEPARTAMENTO'])

    if lat is not None and lon is not None:
        # Crear contenido del popup
        popup_content = f"""
        <h4>Datos del Departamento</h4>
        <p><b>Nombre del Departamento:</b> {row['DEPARTAMENTO']}</p>
        <p><b>Cantidad de colegios (Calendario A):</b> {row['Colegios_Calendario_A']}</p>
        <p><b>Cantidad de colegios (Calendario B):</b> {row['Colegios_Calendario_B']}</p>
        <p><b>Total de Matrículas:</b> {row['TOTAL_MATRICULA']}</p>
        <p><b>Cantidad de Sedes:</b> {row['CANTIDAD_SEDES']}</p>
        """
        iframe = folium.IFrame(html=popup_content, width=300, height=200)
        popup = folium.Popup(iframe, max_width=2650)

        # Crear marcador con popup
        folium.Marker(
            location=[lat, lon],
            popup=popup,
            icon=folium.Icon(color='blue' if row['Colegios_Calendario_A'] > 0 else 'green')
        ).add_to(mapa)

# Mostrar el mapa
mapa


Index(['DPTO', 'AREA', 'PERIMETER', 'HECTARES', 'geometry', 'HORARIO',
       'CANTIDAD'],
      dtype='object')
Index(['DPTO', 'NOMBRE_DPT', 'AREA', 'PERIMETER', 'HECTARES', 'geometry'], dtype='object')
